In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



### Alternative Portfolio Analysis (created by CCM)
- We are choosing to calculate beta using the adjusted close prices, not close prices
- This creates a discrepancy between the beta computed here and the beta observed on yahoofinance

In [ ]:
# Download data for CTA, SPY, QIS, DBMF, SVOL, FIG, QLENX, AQMNX
data = yf.download(["CTA", "SPY", "DBMF", "SVOL", "FIG", "QIS", "QLENX", "AQMNX"], 
                  start="2019-01-01", 
                  period="1mo", 
                  auto_adjust=False)  # Using auto_adjust=True to get adjusted prices

# Get adjusted close prices
adj_close = data['Adj Close']

# Compute returns and log returns
returns = adj_close.pct_change()
log_returns = np.log(adj_close/adj_close.shift(1))

# Calculate correlations for both returns and log returns
return_correlations = returns.corr()
log_return_correlations = log_returns.corr()

# Create heatmaps for returns
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.heatmap(return_correlations, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Regular Returns Correlations')

plt.subplot(1, 2, 2)
sns.heatmap(log_return_correlations, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Log Returns Correlations')

plt.tight_layout()
plt.show()

# Create dendrograms
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

sns.clustermap(return_correlations, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Hierarchical Clustering - Regular Returns')

sns.clustermap(log_return_correlations, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Hierarchical Clustering - Log Returns')

plt.tight_layout()
plt.show()

# Calculate statistics for both return types
stats = pd.DataFrame({
    'Mean Return': returns.mean(),
    'Return Std': returns.std(),
    'Log Mean Return': log_returns.mean(),
    'Log Return Std': log_returns.std(),
    'Sharpe Ratio (Returns)': returns.mean() / returns.std() * np.sqrt(252),  # Annualized Sharpe
    'Sharpe Ratio (Log Returns)': log_returns.mean() / log_returns.std() * np.sqrt(252)  # Annualized Sharpe
})

print("\nAsset Statistics:")
print(stats)

In [ ]:
# Large-cap: VV, VUG, VTV
# Mid-cap: VO, VOT,VOE
# Small-cap: VBR, VBK, VBR
# Intl Developed: VEA
# Intl Emerging: VWO


# Define tickers for different market segments
tickers = ['VV', 'VUG', 'VTV', 'VO', 'VOT', 'VOE', 'VBR', 'VBK', 'VEA', 'VWO', 'SPY']

# Download data
data = yf.download(tickers, start='2010-01-01', auto_adjust=False)['Adj Close']

# Calculate returns
segment_returns = data.pct_change()

# Calculate correlation matrix
segment_correlations = segment_returns.corr()

# Create heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(segment_correlations, annot=True, cmap='coolwarm', vmin=0.5, vmax=1)
plt.title('Market Segment Return Correlations')
plt.tight_layout()
plt.show()
# Create dendrogram using hierarchical clustering
plt.figure(figsize=(12, 10))
sns.clustermap(segment_correlations, annot=True, cmap='coolwarm', vmin=0.5, vmax=1)
plt.title('Hierarchical Clustering of Market Segments')
plt.tight_layout()
plt.show()


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Portfolio Definition
portfolio = {
    'ACWX': 0.1412,
    'DFFVX': 0.0306,
    'FSMDX': 0.0408,
    'FXAIX': 0.3200,
    'FXNAX': 0.2599,
    'PRNHX': 0.0230,
    'RGAGX': 0.1700,
    'RSEGX': 0.0209
}

# Verify weights sum to approximately 1
total_weight = sum(portfolio.values())
print(f"Total portfolio weight: {total_weight:.4f}")
if not 0.99 <= total_weight <= 1.01:
    raise ValueError(f"Portfolio weights sum to {total_weight}, should be close to 1.0")

# 2. Data Download
try:
    data = yf.download(
        list(portfolio.keys()),
        start="2019-01-01",
        end="2024-01-31",
        auto_adjust=True
    )
    
    print("\nData shape:", data.shape)
    print("\nFirst few dates of data:")
    print(data['Close'].head())
    
except Exception as e:
    print(f"Error downloading data: {e}")
    raise

# 3. Portfolio Analysis
# Calculate daily returns
daily_returns = data['Close'].pct_change()

# Calculate portfolio returns
portfolio_returns = pd.Series(0, index=daily_returns.index)
for asset, weight in portfolio.items():
    portfolio_returns += daily_returns[asset] * weight

# Calculate cumulative returns
portfolio_cumulative = (1 + portfolio_returns).cumprod()

# 4. Performance Metrics
metrics = {
    'Annual Return': portfolio_returns.mean() * 252,
    'Annual Volatility': portfolio_returns.std() * np.sqrt(252),
    'Sharpe Ratio': (portfolio_returns.mean() * 252) / (portfolio_returns.std() * np.sqrt(252)),
    'Max Drawdown': (portfolio_cumulative / portfolio_cumulative.cummax() - 1).min(),
    'Skewness': portfolio_returns.skew(),
    'Kurtosis': portfolio_returns.kurtosis()
}

print("\nPortfolio Performance Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# 5. Visualizations
# Plot 1: Cumulative Returns
plt.figure(figsize=(12, 6))
portfolio_cumulative.plot()
plt.title('Portfolio Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Growth of $1')
plt.grid(True)
plt.show()

# Plot 2: Monthly Returns Heatmap
monthly_returns = portfolio_returns.resample('M').agg(lambda x: (1 + x).prod() - 1)
monthly_returns_table = pd.DataFrame(monthly_returns)
monthly_returns_table.index = pd.MultiIndex.from_arrays([
    monthly_returns.index.year,
    monthly_returns.index.month
])
monthly_returns_pivot = monthly_returns_table.pivot_table(
    index=monthly_returns_table.index.get_level_values(1),
    columns=monthly_returns_table.index.get_level_values(0)
)

plt.figure(figsize=(12, 6))
sns.heatmap(monthly_returns_pivot, annot=True, fmt='.2%', cmap='RdYlGn', center=0)
plt.title('Monthly Returns Heatmap')
plt.show()

# Plot 3: Rolling Metrics
window = 252  # One year rolling window
rolling_metrics = pd.DataFrame({
    'Rolling Annual Return': portfolio_returns.rolling(window).mean() * 252,
    'Rolling Annual Volatility': portfolio_returns.rolling(window).std() * np.sqrt(252),
    'Rolling Sharpe Ratio': (portfolio_returns.rolling(window).mean() * 252) / 
                           (portfolio_returns.rolling(window).std() * np.sqrt(252))
})

fig, axes = plt.subplots(3, 1, figsize=(12, 12))
fig.suptitle('Rolling Portfolio Metrics (1-Year Window)')

rolling_metrics['Rolling Annual Return'].plot(ax=axes[0])
axes[0].set_title('Rolling Annual Return')
axes[0].grid(True)

rolling_metrics['Rolling Annual Volatility'].plot(ax=axes[1])
axes[1].set_title('Rolling Annual Volatility')
axes[1].grid(True)

rolling_metrics['Rolling Sharpe Ratio'].plot(ax=axes[2])
axes[2].set_title('Rolling Sharpe Ratio')
axes[2].grid(True)

plt.tight_layout()
plt.show()

# 6. Asset-Level Analysis
asset_metrics = pd.DataFrame({
    'Weight': portfolio.values(),
    'Annual Return': daily_returns.mean() * 252,
    'Annual Volatility': daily_returns.std() * np.sqrt(252),
    'Sharpe Ratio': (daily_returns.mean() * 252) / (daily_returns.std() * np.sqrt(252))
})

print("\nIndividual Asset Metrics:")
print(asset_metrics)

# 7. Correlation Analysis
correlation_matrix = daily_returns.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Asset Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Portfolio Definition
target_portfolio = {
    'ACWX': 0.1412,
    'DFFVX': 0.0306,
    'FSMDX': 0.0408,
    'FXAIX': 0.3200,
    'FXNAX': 0.2599,
    'PRNHX': 0.0230,
    'RGAGX': 0.1700,
    'RSEGX': 0.0209
}

# 2. Data Download
try:
    data = yf.download(
        list(target_portfolio.keys()),
        start="2019-01-01",
        end="2024-12-30",
        auto_adjust=False
    )
    print("\nData shape:", data.shape)
except Exception as e:
    print(f"Error downloading data: {e}")
    raise

# 3. Portfolio Simulation with Rebalancing
def simulate_portfolio(prices, target_weights, rebalance_frequency='Q'):
    """
    Simulate portfolio with periodic rebalancing
    rebalance_frequency: 'Q' for quarterly
    """
    # Initialize portfolio
    portfolio_value = 1.0  # Start with $1
    current_weights = target_weights.copy()
    shares = {asset: portfolio_value * weight / prices['Adj Close'][asset][0] 
             for asset, weight in target_weights.items()}
    
    # Track portfolio evolution
    portfolio_values = []
    actual_weights = []
    rebalance_dates = []
    
    # Iterate through dates
    for date in prices.index:
        # Calculate current portfolio value and weights
        current_value = sum(shares[asset] * prices['Adj Close'][asset][date] 
                          for asset in shares)
        current_weight = {asset: shares[asset] * prices['Adj Close'][asset][date] / current_value 
                        for asset in shares}
        
        portfolio_values.append(current_value)
        actual_weights.append(current_weight)
        
        # Check if rebalancing is needed (end of quarter)
        if (date.month % 3 == 0) and (date.day >= 28):
            shares = {asset: current_value * target_weights[asset] / prices['Adj Close'][asset][date] 
                     for asset in target_weights}
            rebalance_dates.append(date)
    
    return pd.Series(portfolio_values, index=prices.index), pd.DataFrame(actual_weights, index=prices.index), rebalance_dates

# Run simulation
portfolio_values, actual_weights, rebalance_dates = simulate_portfolio(data, target_portfolio)

# 4. Calculate Returns and Metrics
portfolio_returns = portfolio_values.pct_change()

metrics = {
    'Annual Return': portfolio_returns.mean() * 252,
    'Annual Volatility': portfolio_returns.std() * np.sqrt(252),
    'Sharpe Ratio': (portfolio_returns.mean() * 252) / (portfolio_returns.std() * np.sqrt(252)),
    'Max Drawdown': (portfolio_values / portfolio_values.cummax() - 1).min(),
    'Number of Rebalances': len(rebalance_dates)
}

print("\nPortfolio Performance Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# 5. Visualizations
# Plot 1: Portfolio Value Over Time
plt.figure(figsize=(12, 6))
portfolio_values.plot()
for date in rebalance_dates:
    plt.axvline(x=date, color='r', linestyle='--', alpha=0.2)
plt.title('Portfolio Value Over Time\n(Red lines indicate rebalancing dates)')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.grid(True)
plt.show()

# Plot 2: Asset Allocation Over Time
plt.figure(figsize=(12, 6))
actual_weights.plot.area()
plt.title('Asset Allocation Over Time')
plt.xlabel('Date')
plt.ylabel('Weight')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Plot 3: Tracking Error from Target Weights
tracking_error = pd.DataFrame({
    asset: np.abs(actual_weights[asset] - target_weight)
    for asset, target_weight in target_portfolio.items()
})

plt.figure(figsize=(12, 6))
tracking_error.plot()
plt.title('Tracking Error from Target Weights')
plt.xlabel('Date')
plt.ylabel('Absolute Deviation from Target Weight')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Print rebalancing dates and maximum deviations
print("\nRebalancing Dates:")
for date in rebalance_dates:
    print(date.strftime('%Y-%m-%d'))

print("\nMaximum Weight Deviations:")
max_deviations = {asset: tracking_error[asset].max() 
                 for asset in target_portfolio.keys()}
for asset, dev in max_deviations.items():
    print(f"{asset}: {dev:.4%}")

# Plot 4: Rolling Metrics
fig, axes = plt.subplots(3, 1, figsize=(12, 12))
fig.suptitle('Rolling Portfolio Metrics')

# Rolling Returns
window = 252  # 1 year of trading days
rolling_returns = portfolio_returns.rolling(window).mean() * 252
rolling_returns.plot(ax=axes[0])
axes[0].set_title('Rolling 1-Year Returns')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Annual Return')
axes[0].grid(True)

# Rolling Volatility 
rolling_vol = portfolio_returns.rolling(window).std() * np.sqrt(252)
rolling_vol.plot(ax=axes[1])
axes[1].set_title('Rolling 1-Year Volatility')
axes[1].set_xlabel('Date') 
axes[1].set_ylabel('Annual Volatility')
axes[1].grid(True)

# Rolling Sharpe Ratio
risk_free_rate = 0.02  # Assuming 2% risk-free rate
rolling_sharpe = (rolling_returns - risk_free_rate) / rolling_vol
rolling_sharpe.plot(ax=axes[2])
axes[2].set_title('Rolling 1-Year Sharpe Ratio')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Sharpe Ratio')
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Download SPY data for the same period
spy_data = yf.download('SPY', start="2019-01-01", end="2024-12-30", auto_adjust=False)
spy_returns = spy_data['Adj Close'].pct_change()

# Calculate cumulative returns for both portfolio and SPY
portfolio_cum_returns = (1 + portfolio_returns).cumprod()
spy_cum_returns = (1 + spy_returns).cumprod()

# Plot cumulative returns comparison
plt.figure(figsize=(12, 6))
plt.plot(portfolio_cum_returns, label='Portfolio', linewidth=2)
plt.plot(spy_cum_returns, label='SPY', alpha=0.7, linestyle='--')
plt.title('Portfolio vs SPY Performance')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Calculate and print performance metrics
# print("\nPerformance Comparison:")
# print(f"Portfolio Total Return: {portfolio_cum_returns[-1]-1:.2%}")
# print(f"SPY Total Return: {spy_cum_returns[-1]-1:.2%}")
# print(f"\nPortfolio Volatility: {portfolio_returns.std() * np.sqrt(252):.2%}")
# print(f"SPY Volatility: {spy_returns.std() * np.sqrt(252):.2%}")

